# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>1 |</span></b> <b>INTRODUCTION</b></div>

👋 Welcome to "🧠Exploring EEG: A Beginner's Guide"! 

If you're fascinated by the wonders of the human brain and the intricate patterns of brainwaves, but find the world of Electroencephalography (EEG) analysis daunting, you're in the right place. 

This notebook is designed for beginners like me & you, aiming to demystify the complexities of EEG data and make your learning journey both enjoyable and informative.

### <b><span style='color:#FFCE30'> 1.1 |</span> Intention of the notebook</b>
In this notebook, we will embark on an exploratory journey into the realm of EEG data analysis. Our goal is to provide a clear, step-by-step guide to understanding and analyzing EEG signals, which are crucial in detecting and classifying brain activities, such as seizures. We aim to:

* Break down complex concepts into easily digestible sections.
* Illustrate each step with practical code examples.
* Reference public notebooks and discussions to enhance your learning experience.


### <b><span style='color:#FFCE30'> 1.2 |</span> Learning Objective</b>
By the end of this notebook, you will have a foundational understanding of:

* The basics of EEG signals and their significance in medical research and neurology.
* How to preprocess and analyze EEG data.
* Run through the basic code to build a machine learning model for EEG data classification.

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>2 |</span></b> <b>REFERENCE & ACKNOWLEDGEMENT</b></div>

This notebook wouldn't be possible without the valuable insights and contributions from the Kaggle community. I've leveraged several resources to compile the most effective learning path for us:

* https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-8
* https://www.kaggle.com/code/mvvppp/hms-eda-and-domain-journey
* https://www.kaggle.com/code/ksooklall/hms-banana-montage
* https://www.kaggle.com/code/mpwolke/seizures-classification-parquet


Feel free to explore these resources alongside this notebook to deepen your understanding.

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>3 |</span></b> <b>LOAD LIBARIES</b></div>

In [1]:
import os
import pandas as pd, numpy as np
from glob import glob
import matplotlib.pyplot as plt
VER = 1

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>4 |</span></b> <b>INTRODUCTION TO EEG AND SEIZURE DETECTION</b></div>

<b><span style='color:#FFCE30'> 4.1 |</span> Electroencephalography (EEG) - The Window into Brain Activity</b>

* Electroencephalography, commonly known as EEG, is a non-invasive method used by medical professionals to record electrical activity in the brain. 
* This is done using electrodes placed along the scalp. 
* EEG is a crucial tool in diagnosing neurological disorders, especially epilepsy, which is characterized by recurrent seizures.

<img src="https://www.researchgate.net/profile/Sebastian-Nagel-4/publication/338423585/figure/fig1/AS:844668573073409@1578396089381/Sketch-of-how-to-record-an-Electroencephalogram-An-EEG-allows-measuring-the-electrical.png" alt="EEG" width="600" height="400">



In [2]:
# check the reading of one parquet for understanding

BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'

df = pd.DataFrame({'path': glob(BASE_PATH + '**/*.parquet')})
df['test_type'] = df['path'].str.split('/').str.get(-2).str.split('_').str.get(-1)
df['id'] = df['path'].str.split('/').str.get(-1).str.split('.').str.get(0)

df_eeg = pd.read_parquet(BASE_PATH + 'train_eegs/1000913311.parquet')
df_eeg.head()

,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-105.849998,-89.230003,-79.459999,-49.230000,-99.730003,-87.769997,-53.330002,-50.740002,-32.250000,-42.099998,-43.270000,-88.730003,-74.410004,-92.459999,-58.930000,-75.739998,-59.470001,8.210000,66.489998,1404.930054
1,-85.470001,-75.070000,-60.259998,-38.919998,-73.080002,-87.510002,-39.680000,-35.630001,-76.839996,-62.740002,-43.040001,-68.629997,-61.689999,-69.320000,-35.790001,-58.900002,-41.660000,196.190002,230.669998,3402.669922
2,8.840000,34.849998,56.430000,67.970001,48.099998,25.350000,80.250000,48.060001,6.720000,37.880001,61.000000,16.580000,55.060001,45.020000,70.529999,47.820000,72.029999,-67.180000,-171.309998,-3565.800049
3,-56.320000,-37.279999,-28.100000,-2.820000,-43.430000,-35.049999,3.910000,-12.660000,8.650000,3.830000,4.180000,-51.900002,-21.889999,-41.330002,-11.580000,-27.040001,-11.730000,-91.000000,-81.190002,-1280.930054
4,-110.139999,-104.519997,-96.879997,-70.250000,-111.660004,-114.430000,-71.830002,-61.919998,-76.150002,-79.779999,-67.480003,-99.029999,-93.610001,-104.410004,-70.070000,-89.250000,-77.260002,155.729996,264.850006,4325.370117


In [3]:
# Determine the number of channels
# Assuming each row is a time point and each column is a channel
n_channels = df_eeg.shape[1]
n_channels

20

* The headers in the dataset (Fp1, F3, C3, P3, F7, T3, T5, O1, Fz, Cz, Pz, Fp2, F4, C4, P4, F8, T4, T6, O2, EKG) are standard electrode placement labels used in electroencephalography (EEG). 
* These labels correspond to specific positions on the scalp where EEG electrodes are placed to record brain activity. 
* Here's a brief overview of what they represent:

1. **Fp1, Fp2:** Frontopolar electrodes, located on the forehead, left and right side.
2. **F3, F4:** Frontal electrodes, on the left and right side of the forehead.
3. **C3, C4:** Central electrodes, placed above the left and right hemispheres of the brain.
4. **P3, P4:** Parietal electrodes, located on the upper back portion of the head, left and right sides.
5. **O1, O2:** Occipital electrodes, positioned at the back of the head near the visual cortex.
6. **T3, T4, T5, T6:** Temporal electrodes, situated on the left and right sides of the head near the ears. They are often involved in monitoring auditory functions.
7. **F7, F8:** Frontal-temporal electrodes, located at the front of the temporal lobes.
8. **Fz, Cz, Pz:** Midline electrodes, located at the frontal (Fz), central (Cz), and parietal (Pz) positions on the midline of the head.
9. **EKG:** Electrocardiogram electrode, which records the heart’s electrical activity. It's not directly related to brain activity but can be important in some EEG analyses.


<img src="https://www.researchgate.net/profile/Danny-Plass-Oude-Bos/publication/237777779/figure/fig3/AS:669556259434497@1536646060035/10-20-system-of-electrode-placement.png" alt="10-20-system-of-electrode-placement" width="300" height="150">

<b><span style='color:#FFCE30'> 4.2 |</span> Seizures and Their Impact</b>
* Seizures are sudden, uncontrolled electrical disturbances in the brain that can cause changes in behavior, feelings, movements, and levels of consciousness. 
* Detecting and classifying seizures accurately is vital for appropriate treatment and care, especially in critically ill patients.

<b><span style='color:#FFCE30'> 4.3 |</span> The Challenge of Manual EEG Analysis</b>

* Traditionally, EEG data analysis relies on visual inspection by trained neurologists. 
* This process is not only time-consuming and labor-intensive but also prone to errors due to fatigue and subjective interpretation.

<img src="https://slideplayer.com/slide/12925171/78/images/2/Manual+Interpretation+of+EEGs.jpg" alt="Manual Interpretation of EEG" width="700" height="300">
Source: Automated Identification of Abnormal Adult EEG, S. López, G. Suarez, D. Jungreis, I. Obeid and J. Picone, Neural Engineering Data Consortium, Temple University


<b><span style='color:#FFCE30'> 4.4 |</span> The Role of Data Science in EEG Analysis</b>

* Automating EEG Interpretation
The advent of machine learning and data science offers an opportunity to automate the interpretation of EEG data. By developing algorithms that can detect and classify different patterns in EEG signals, we can aid neurologists in making faster, more accurate diagnoses.

* The Data Science Approach
Data scientists approach this challenge by first preprocessing the EEG data, which involves filtering out noise and extracting relevant features. Machine learning models are then trained on these features to distinguish between different types of brain activity.

<img src="https://www.researchgate.net/profile/Huiguang-He/publication/336336651/figure/fig1/AS:834361356197888@1575938657076/The-flow-chart-of-EEG-emotion-classification-with-similarity-learning-network.png" alt="flowchart for EEG classification" width="700" height="300">


<b><span style='color:#FFCE30'> 4.5 |</span> Understanding EEG Patterns</b>

In the realm of EEG analysis for seizure detection, certain patterns are of particular interest:

1. **Seizure (SZ):** Characterized by abnormal rhythmic activity, indicative of a seizure.
2. **Generalized Periodic Discharges (GPD):** Patterns that may be seen in various encephalopathies.
3. **Lateralized Periodic Discharges (LPD):** Often associated with focal brain lesions.
4. **Lateralized Rhythmic Delta Activity (LRDA):** Can be observed in focal brain dysfunction.
5. **Generalized Rhythmic Delta Activity (GRDA):** Typically related to diffuse brain dysfunction.
6. **"Other" Patterns:** Any other type of activity not falling into the above categories.

<b><span style='color:#FFCE30'> 4.6 |</span> Interpreting Complex EEG Data</b>

EEG data interpretation can be complex, especially in edge cases where expert neurologists may not agree on a classification. This is where machine learning models can particularly shine by providing an additional layer of analysis.

<img src="https://www.neurology.org/cms/10.1212/WNL.0000000000207127/asset/bd84c182-712c-41ab-8742-cecf9d49a322/assets/images/large/5ff2.jpg" alt="flowchart for EEG classification" width="700" height="300">

Source: Development of Expert-Level Classification of Seizures and Rhythmic and Periodic Patterns During EEG Interpretation https://www.neurology.org/doi/10.1212/WNL.0000000000207127


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>5 |</span></b> <b>LOAD TRAIN DATA</b></div>

In [4]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>6 |</span></b> <b>CREATE NON-OVERLAPPING EEG ID TRAIN DATA</b></div>

Following the notebook from Chris Deotte: https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-8,
Initial discussion found here https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

We perform the following because:

* **Match Training Data with Test Data Format:** The competition states that the test data does not have multiple segments from the same eeg_id. To make the training data similar to the test data, we also use only one segment per eeg_id in the training data.

* **Remove Redundancies:** This approach ensures that the training data does not have overlapping or redundant information, which can lead to a more accurate and generalizable machine learning model.

* **Consistency in Data:** By standardizing how we handle the EEG segments in training, we ensure that our model learns from data that is consistent in format with the data it will be tested on.

* **Data Preparation for Machine Learning:** The normalization of target variables and inclusion of relevant features like patient_id and expert_consensus prepare the dataset for effective machine learning modeling.

In [5]:
# Creating a Unique EEG Segment per eeg_id:
# The code groups (groupby) the EEG data (df) by eeg_id. Each eeg_id represents a different EEG recording.
# It then picks the first spectrogram_id and the earliest (min) spectrogram_label_offset_seconds for each eeg_id. This helps in identifying the starting point of each EEG segment.
# The resulting DataFrame train has columns spec_id (first spectrogram_id) and min (earliest spectrogram_label_offset_seconds).
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']


# Finding the Latest Point in Each EEG Segment:
# The code again groups the data by eeg_id and finds the latest (max) spectrogram_label_offset_seconds for each segment.
# This max value is added to the train DataFrame, representing the end point of each EEG segment.
tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp


tmp = df.groupby('eeg_id')[['patient_id']].agg('first') # The code adds the patient_id for each eeg_id to the train DataFrame. This links each EEG segment to a specific patient.
train['patient_id'] = tmp


tmp = df.groupby('eeg_id')[TARGETS].agg('sum') # The code sums up the target variable counts (like votes for seizure, LPD, etc.) for each eeg_id.
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values # It then normalizes these counts so that they sum up to 1. This step converts the counts into probabilities, which is a common practice in classification tasks.
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first') # For each eeg_id, the code includes the expert_consensus on the EEG segment's classification.
train['target'] = tmp

train = train.reset_index() # This makes eeg_id a regular column, making the DataFrame easier to work with.
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>7 |</span></b> <b>FEATURE ENGINEERING</b></div>



<b><span style='color:#FFCE30'> 7.1 |</span> 10 min and 20 sec windows</b>

* The code belows efficiently reads spectrogram data, from a single combined file, based on the set variable. We relied on the dataset by Chris Deotte to save time. https://www.kaggle.com/datasets/cdeotte/brain-spectrograms
* It then performs feature engineering by calculating mean and minimum values over two different time windows for each frequency in the spectrogram.
It produce produces in 1600 features (400 features × 4 calculations) for each EEG ID.
* The new features are intended to help the model better understand and classify the EEG data.
* This approach is designed to enhance the model's performance by providing it with more detailed information derived from the spectrogram data.

In [6]:
READ_SPEC_FILES = False # If READ_SPEC_FILES is False, the code reads the combined file instead of individual files.
FEATURE_ENGINEER = True
READ_EEG_SPEC_FILES = False

In [7]:
%%time
# READ ALL SPECTROGRAMS
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: user 2.78 s, sys: 6.04 s, total: 8.82 s
Wall time: 59.9 s


# Autoencoder Definition

<b><span style='color:#FFCE30'> 7.2 |</span>  Frequency Band Analysis</b>

#### Frequency Band Feature Extraction:

* The function extract_frequency_band_features is designed to process a segment of EEG data. EEG data is a complex signal that represents the electrical activity of the brain.
* This function divides the EEG signal into different frequency bands: Delta, Theta, Alpha, Beta, and Gamma. These bands are significant in neuroscientific studies as they are associated with different brain states and activities.

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128044902000026-f02-01-9780128044902.jpg)


1. **Delta (0.5 – 4 Hz):**
Delta waves are the slowest brainwaves and are typically associated with deep sleep and restorative processes in the body. They are most prominent during dreamless sleep and play a role in healing and regeneration.
2. **Theta (4 – 8 Hz):**
Theta waves occur during light sleep, deep meditation, and REM (Rapid Eye Movement) sleep. They are linked to creativity, intuition, daydreaming, and fantasizing. Theta states are often associated with subconscious mind activities.
3. **Alpha (8 – 12 Hz):**
Alpha waves are present during physically and mentally relaxed states but still alert. They are typical in wakeful states that involve a relaxed and effortless alertness. Alpha waves aid in mental coordination, calmness, alertness, and learning.
4. **Beta (12 – 30 Hz):**
Beta waves dominate our normal waking state of consciousness when attention is directed towards cognitive tasks and the outside world. They are associated with active, busy or anxious thinking and active concentration.
5. **Gamma (30 – 45 Hz):**
Gamma waves are involved in higher mental activity and consolidation of information. They are important for learning, memory, and information processing. Gamma waves are thought to be the fastest brainwave frequency and relate to simultaneous processing of information from different brain areas.




* For each frequency band, the function applies a bandpass filter to isolate that band's signal. It then computes statistical features (mean, standard deviation, maximum, and minimum) for each band, effectively capturing the characteristics of the EEG signal in these different frequency ranges.
* The use of np.nanmean, np.nanstd, np.nanmax, and np.nanmin ensures that the calculations are robust to NaN (Not a Number) values in the data, which might occur due to various reasons like signal loss or artifacts.

#### Feature Aggregation and PCA:

* The main script initializes a Principal Component Analysis (PCA) model with the intention of reducing the dimensionality of the extracted features. PCA is a common technique used to transform high-dimensional datasets into a lower-dimensional space while retaining most of the variance in the data.
* The script iterates over rows in the train dataset, extracting EEG segments and applying the extract_frequency_band_features function to each channel in these segments. The extracted features from all channels are aggregated.
* However, before applying PCA, any NaN values in the aggregated data (data_original) are handled using mean imputation. This step ensures that the PCA algorithm, which cannot handle NaN values, receives a clean dataset.
* After imputation, PCA is applied to transform the features into a principal component space, and these transformed features are added back into the train DataFrame.
* This process ultimately results in a feature set that's potentially more informative and concise for machine learning models, helping in tasks like classification or anomaly detection in EEG data.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.init as init
import torch.nn.init as init
import gc
gc.collect()
torch.cuda.empty_cache()

"""
Ideas To Prevent Loss Nans
1. Normalize Data Better
2. Less Deep / Wide Architecture
3. CNN instead of FCNN
"""
class AE(torch.nn.Module):
    def __init__(self, numFrequencies, numRows, numFeatures=100):
        super().__init__()

        # Building a linear encoder with Batch Normalization
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(numFrequencies * numRows, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, numFeatures),
            torch.nn.ReLU(),
        )

        # Building a linear decoder with Batch Normalization
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(numFeatures, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, numFrequencies * numRows),
            torch.nn.Sigmoid()
        )

        # Apply Xavier initialization to the weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.xavier_uniform_(m.weight)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [9]:
alpha_frequencies = 21 * 4
delta_frequencies = 18 * 4
theta_frequencies = 20 * 4
beta_frequencies = 41 * 4

# Autoencoder Feature Engineering - Spectrogram Level

In [10]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')
SPEC_FREQS = len(pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:])
print(f"Num Frequencies: {SPEC_FREQS}")
numFeatures = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu') # delete when issue resolved

print("Using: ", device)
"""
Define alpha feature autoencoder
"""
model_alpha = AE(alpha_frequencies, 300, numFeatures=numFeatures)
model_alpha = model_alpha.to(device)
loss_function_alpha = torch.nn.MSELoss()
optimizer_alpha = torch.optim.Adam(model_alpha.parameters(),
                            lr = 3e-4,
                            )

"""
Define beta feature autoencoder
"""
model_beta = AE(beta_frequencies, 300, numFeatures=numFeatures)
model_beta = model_beta.to(device)
loss_function_beta = torch.nn.MSELoss()
optimizer_beta = torch.optim.Adam(model_beta.parameters(),
                            lr = 3e-4,
                            )

"""
Define delta feature autoencoder
"""
model_delta = AE(delta_frequencies, 300, numFeatures=numFeatures)
model_delta = model_delta.to(device)
loss_function_delta = torch.nn.MSELoss()
optimizer_delta = torch.optim.Adam(model_delta.parameters(),
                            lr = 3e-4,
                            )


"""
Define theta feature autoencoder
"""
model_theta = AE(theta_frequencies, 300, numFeatures=numFeatures)
model_theta = model_theta.to(device)
loss_function_theta = torch.nn.MSELoss()
optimizer_theta = torch.optim.Adam(model_theta.parameters(),
                            lr = 3e-4,
                            )

# No gamma waves in this data!

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.7 µs
Num Frequencies: 400
Using:  cuda


In [11]:
from scipy import signal
def extract_frequency_band_features(segment):
    
    cols = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:] # like LR_14.32
    channel_groups = ['LL', 'RL', 'LP', 'RP']
    
    eeg_bands = {'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30)}
    band_datapoints = {
        "Alpha": [],
        "Delta": [],
        "Theta": [],
        "Beta": [],
    }
    
    for channel_group in channel_groups:
        for band in eeg_bands:
            low, high = eeg_bands[band]
            # Filter signal for the specific band
            idxs = []
            for idx, col in enumerate(cols):
                if channel_group in col and float(col.split("_")[1]) <= high and float(col.split("_")[1]) >= low:
                    idxs.append(idx)
                        
            filtered = segment[:, idxs].flatten()
            band_datapoints[band].append(filtered)
    
    for band in band_datapoints:
        band_datapoints[band] = np.array(band_datapoints[band]).flatten() 
        # join all 4 group signals into one to reconstruct in autoencoder
    return band_datapoints
            

In [12]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.impute import SimpleImputer
torch.autograd.set_detect_anomaly(True)

# Create a SimpleImputer instance
nan_imputer = SimpleImputer(strategy='mean')
log_noise = 1
batch_size = 100
print(f"Training Autoencoder on {len(train)} datapoints with batch size {batch_size}")
num_epochs = 8 # fine with 2-3 epochs but should do more with GPU if possible
for epoch in range(num_epochs): 
    num_batches = len(train) // batch_size + 1
    epoch_loss_alpha = 0.0
    epoch_loss_beta = 0.0
    epoch_loss_delta = 0.0
    epoch_loss_theta = 0.0
    
    print(f"Batches {num_batches}:", end=' ')
    
    for i in range(num_batches):        
        
        row_start = i * batch_size
        row_end = min((i + 1) * batch_size, len(train))
        
        input_alpha_list = []
        input_beta_list = []
        input_delta_list = []
        input_theta_list = []
        
        for k in range(row_start, row_end):
            row = train.iloc[k]
            r = int((row['min'] + row['max']) // 4)
            
            eeg_segment = spectrograms[row.spec_id][r:r+300, :]
            
            signals = extract_frequency_band_features(eeg_segment)

            for band, sigs in signals.items():
                temp = np.where(np.isfinite(sigs), sigs, np.nan)
                signals[band] = nan_imputer.fit_transform([temp])[0]
            
            # Convert to torch tensors and append to the lists flattened since VNN!!!
            if len(signals["Alpha"]) == alpha_frequencies * 300:
                vals = signals["Alpha"]
                vals = np.log(vals + log_noise)
                norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
                input_alpha_list.append(norm_vals)
            else:
                pass
                # print("Alpha mismatch shape of: ", len(signals["Alpha"].flatten()))
                
            if len(signals["Beta"]) == beta_frequencies * 300:
                vals = signals["Beta"]
                vals = np.log(vals + log_noise)
                norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
                input_beta_list.append(norm_vals)
            else:
                pass
                # print("Beta mismatch shape of: ", len(signals["Beta"].flatten()))
                
            if len(signals["Delta"]) == delta_frequencies * 300:
                vals = signals["Delta"]
                vals = np.log(vals + log_noise)
                norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
                input_delta_list.append(norm_vals)
            else:
                pass
                # print("Delta mismatch shape of: ", len(signals["Delta"].flatten()))
                
            if len(signals["Theta"]) == theta_frequencies * 300:
                vals = signals["Theta"]
                vals = np.log(vals + log_noise)
                norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
                input_theta_list.append(norm_vals)
            else:
                pass 
                # print("Theta mismatch shape of: ", len(signals["Theta"].flatten()))
                
                
        # Forward pass through the autoencoders
        input_alpha_batch = torch.tensor(input_alpha_list, dtype=torch.float32).to(device)
        input_beta_batch = torch.tensor(input_beta_list, dtype=torch.float32).to(device)
        input_delta_batch = torch.tensor(input_delta_list, dtype=torch.float32).to(device)
        input_theta_batch = torch.tensor(input_theta_list, dtype=torch.float32).to(device)        
        
        output_alpha_batch = model_alpha(input_alpha_batch)
        output_beta_batch = model_beta(input_beta_batch)
        output_delta_batch = model_delta(input_delta_batch)
        output_theta_batch = model_theta(input_theta_batch)

        # Calculate loss and perform optimization for alpha autoencoder
        loss_alpha = loss_function_alpha(output_alpha_batch, input_alpha_batch)
        optimizer_alpha.zero_grad()
        loss_alpha.backward()
        optimizer_alpha.step()

        # Calculate loss and perform optimization for beta autoencoder
        loss_beta = loss_function_beta(output_beta_batch, input_beta_batch)
        optimizer_beta.zero_grad()
        loss_beta.backward()
        optimizer_beta.step()
        
        # Calculate loss and perform optimization for deltaa autoencoder
        loss_delta = loss_function_delta(output_delta_batch, input_delta_batch)
        optimizer_delta.zero_grad()
        loss_delta.backward()
        optimizer_delta.step()
        
        # Calculate loss and perform optimization for theta autoencoder
        loss_theta = loss_function_theta(output_theta_batch, input_theta_batch)
        optimizer_theta.zero_grad()
        loss_theta.backward()
        optimizer_theta.step()

        # Accumulate epoch loss
        epoch_loss_alpha += loss_alpha.item()
        epoch_loss_beta += loss_beta.item()
        epoch_loss_delta += loss_delta.item()
        epoch_loss_theta += loss_theta.item()
        
        # Clean up to avoid memory issues
        del output_alpha_batch, output_beta_batch, output_delta_batch, output_theta_batch
        del input_alpha_batch, input_beta_batch, input_delta_batch, input_theta_batch
        del input_alpha_list, input_beta_list, input_delta_list, input_theta_list
        
        if i % 5 == 0:
            print(f"Done batch {i}", end = '... ')

    # Calculate average loss for the epoch
    avg_loss_alpha = epoch_loss_alpha / num_batches
    avg_loss_beta = epoch_loss_beta / num_batches
    avg_loss_delta = epoch_loss_delta / num_batches
    avg_loss_theta = epoch_loss_theta / num_batches

    print(f"Epoch {epoch} Summary: Avg Loss Alpha: {avg_loss_alpha}, Avg Loss Beta: {avg_loss_beta}, Avg Loss Delta: {avg_loss_delta}, Avg Loss Theta: {avg_loss_theta}")


Training Autoencoder on 17089 datapoints with batch size 100
Batches 171: Done batch 0... Done batch 5... Done batch 10... Done batch 15... Done batch 20... Done batch 25... Done batch 30... Done batch 35... Done batch 40... Done batch 45... Done batch 50... Done batch 55... Done batch 60... Done batch 65... Done batch 70... Done batch 75... Done batch 80... Done batch 85... Done batch 90... Done batch 95... Done batch 100... Done batch 105... Done batch 110... Done batch 115... Done batch 120... Done batch 125... Done batch 130... Done batch 135... Done batch 140... Done batch 145... Done batch 150... Done batch 155... Done batch 160... Done batch 165... Done batch 170... Epoch 0 Summary: Avg Loss Alpha: 0.06385784079892594, Avg Loss Beta: 0.039197452856521854, Avg Loss Delta: 0.02620337721592153, Avg Loss Theta: 0.0308917777623698
Batches 171: Done batch 0... Done batch 5... Done batch 10... Done batch 15... Done batch 20... Done batch 25... Done batch 30... Done batch 35... Done bat

In [13]:
"""
Get Feature Data
"""
print(f"Generating {2 * numFeatures} features on {len(train)} datapoints")
FEATURES = ["feature_{}_alpha".format(i) for i in range(numFeatures)]
FEATURES += ["feature_{}_beta".format(i) for i in range(numFeatures)]
FEATURES += ["feature_{}_delta".format(i) for i in range(numFeatures)]
FEATURES += ["feature_{}_theta".format(i) for i in range(numFeatures)]

data = np.zeros((len(train), len(FEATURES)))

for k in range(len(train)):
    if k%100==0: print(k,', ',end='')
    row = train.iloc[k]
    r = int( (row['min'] + row['max'])//4 )
    
    signals = {
        "Alpha": [], "Beta": [], "Delta": [], "Theta": []
    }
    
    eeg_segment = spectrograms[row.spec_id][r:r+300, :]
    
    signals = extract_frequency_band_features(eeg_segment)

    for band, sigs in signals.items():
        temp = np.where(np.isfinite(sigs), sigs, np.nan)
        signals[band] = nan_imputer.fit_transform([temp])[0]
        
    vals = signals["Alpha"]
    if len(vals) == alpha_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_alpha.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 0:numFeatures] = x
    
    vals = signals["Beta"]
    if len(vals) == beta_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_beta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, numFeatures:2*numFeatures] = x
    
    vals = signals["Delta"]
    if len(vals) == delta_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_delta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 2*numFeatures:3*numFeatures] = x
    
    vals = signals["Theta"]
    if len(vals) == theta_frequencies * 300 * 4:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_theta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 3*numFeatures:4*numFeatures] = x
    

train[FEATURES] = data

print('New train shape:',train.shape)

Generating 400 features on 17089 datapoints
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 1330

In [14]:
from sklearn.preprocessing import StandardScaler

# Columns to be excluded from scaling
excluded_columns = ['eeg_id', 'spec_id', 'min', 'max', 'patient_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote','target']

# Save the columns to be excluded
excluded_data = train[excluded_columns]

# DataFrame with only the columns to be scaled
features = train.drop(columns=excluded_columns)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the features and transform them
features_scaled = scaler.fit_transform(features)

# Create a DataFrame from the scaled features
features_scaled_df = pd.DataFrame(features_scaled, columns=features.columns)

# Concatenate the scaled features with the excluded columns
train_scaled_df = pd.concat([excluded_data.reset_index(drop=True),features_scaled_df,], axis=1)
# train_scaled_df.to_csv("/kaggle/working/")
train_scaled_df 


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,feature_190_theta,feature_191_theta,feature_192_theta,feature_193_theta,feature_194_theta,feature_195_theta,feature_196_theta,feature_197_theta,feature_198_theta,feature_199_theta
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1188113564,0.0,0.0,16610,0.0,0.000000,0.00,0.000000,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17085,4293843368,1549502620,0.0,0.0,15065,0.0,0.000000,0.00,0.000000,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17086,4294455489,2105480289,0.0,0.0,56,0.0,0.000000,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17087,4294858825,657299228,0.0,12.0,4312,0.0,0.000000,0.00,0.000000,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
train_scaled_df.describe()

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,feature_190_theta,feature_191_theta,feature_192_theta,feature_193_theta,feature_194_theta,feature_195_theta,feature_196_theta,feature_197_theta,feature_198_theta,feature_199_theta
count,1.708900e+04,1.708900e+04,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,17089.000000,...,17089.0,17089.0,17089.0,17089.0,17089.0,17089.0,17089.0,17089.0,17089.0,17089.0
mean,2.135226e+09,1.080640e+09,401.650711,431.761191,32839.981977,0.152810,0.142456,0.104062,0.065407,0.114851,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.235712e+09,6.251739e+08,1226.839779,1232.863269,18351.751174,0.331563,0.295541,0.258825,0.187005,0.271425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,5.686570e+05,3.537330e+05,0.000000,0.000000,56.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.062096e+09,5.396648e+08,0.000000,4.000000,17408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,2.123560e+09,1.073264e+09,0.000000,40.000000,32068.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,3.208261e+09,1.641428e+09,308.000000,346.000000,48272.000000,0.000000,0.068966,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,4.294958e+09,2.147388e+09,17556.000000,17632.000000,65494.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>8 |</span></b> <b>TRAIN MODEL</b></div>

* Original work uses catboost, let's try with XGBoost in this version to see the difference in model performance.

In [16]:
import xgboost as xgb
from sklearn.svm import SVC
import gc
from sklearn.model_selection import KFold, GroupKFold
import pickle
from sklearn.multioutput import MultiOutputRegressor
from scipy.special import rel_entr

In [17]:
len(FEATURES)

800

In [18]:
all_oof = []
all_true = []
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
n_splits = 5
gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train_scaled_df, train_scaled_df.target, train_scaled_df.patient_id)):   
    if i >= n_splits:
        continue
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)
    
    # Instantiate the XGBRegressor model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate = 0.1) # uses MSE to predict probabilities

    model = MultiOutputRegressor(xgb_model) # since we have multiple outputs
    
#     model = SVC(probability=True)    
    LABEL_NAMES = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    # Prepare training and validation data
    X_train = train_scaled_df.loc[train_index, FEATURES]
    y_train = train_scaled_df.loc[train_index, LABEL_NAMES]
    X_valid = train_scaled_df.loc[valid_index, FEATURES]
    y_valid = train_scaled_df.loc[valid_index, LABEL_NAMES]
    model.fit(X_train, y_train, verbose=True,) 

    with open(f'XGBoost_f{i}.pkl', 'wb') as f:
        pickle.dump(model, f)

    y_pred = model.predict(X_valid)
    y_pred[y_pred < 0] = 0
    oof = y_pred / np.sum(y_pred, axis=1).reshape(-1,1) # ensure they sum to 1
    true = y_valid.values
    kl_divergence = np.mean(np.sum(true * (np.log(true + 1e-10) - np.log(oof + 1e-10)), axis=1))
    print(f"Kale Divergence: {kl_divergence}")
    
    all_oof.append(oof)
    all_true.append(true)
    
    del X_train, y_train, X_valid, y_valid, oof
    gc.collect()
    
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

#########################
### Fold 1
### train size 13671, valid size 3418
#########################
Kale Divergence: 1.1714089478463006
#########################
### Fold 2
### train size 13671, valid size 3418
#########################
Kale Divergence: 1.2842789893173139
#########################
### Fold 3
### train size 13671, valid size 3418
#########################
Kale Divergence: 1.1626714779324
#########################
### Fold 4
### train size 13671, valid size 3418
#########################
Kale Divergence: 1.160006447936446
#########################
### Fold 5
### train size 13672, valid size 3417
#########################
Kale Divergence: 1.2063009320903042


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>11 |</span></b> <b>INFER TEST</b></div>

In [19]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

Test shape (1, 3)


,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [20]:
%%time
# READ ALL TEST SPECTROGRAMS
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/'
files = os.listdir(PATH2)
print(f'There are {len(files)} spectrogram parquets')

spectrograms_test = {}
for i,f in enumerate(files):
    if i%100==0: print(i,', ',end='')
    tmp = pd.read_parquet(f'{PATH2}{f}')
    name = int(f.split('.')[0])
    spectrograms_test[name] = tmp.iloc[:,1:].values


There are 1 spectrogram parquets
0 , CPU times: user 41.6 ms, sys: 2.2 ms, total: 43.8 ms
Wall time: 43.3 ms


In [21]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')

SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]

TEST_FEATURES = FEATURES

print(f'We are creating {len(TEST_FEATURES)} features for {len(test)} rows... ',end='')


# A data matrix data is initialized to store the new features for each eeg_id in the train DataFrame.
# For each row in train, the code calculates the mean and minimum values within the specified 10-minute and 20-second windows.
# These calculated values are then stored in the data matrix.
# Finally, the matrix is added to the train DataFrame as new columns.

data = np.zeros((len(test),len(TEST_FEATURES)))
for k in range(len(test)):
    if k%100==0: print(k,', ',end='')
    row = test.iloc[k]       
    s = int( row.spectrogram_id )
    spec = pd.read_parquet(f'{PATH2}{s}.parquet')
    
    eeg_segment = spec.iloc[:,1:].values

    signals = extract_frequency_band_features(eeg_segment)

    for band, sigs in signals.items():
        temp = np.where(np.isfinite(sigs), sigs, np.nan)
        signals[band] = nan_imputer.fit_transform([temp])[0]
        
    vals = signals["Alpha"]
    if len(vals) == alpha_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_alpha.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 0:numFeatures] = x
    
    vals = signals["Beta"]
    if len(vals) == beta_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_beta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, numFeatures:2*numFeatures] = x
    
    vals = signals["Delta"]
    if len(vals) == delta_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_delta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 2*numFeatures:3*numFeatures] = x
    
    vals = signals["Theta"]
    if len(vals) == theta_frequencies * 300:
        vals = np.log(vals + log_noise)
        norm_vals = (vals - vals.min()) / (vals.max() - vals.min())
        x = np.array(model_theta.encoder(torch.tensor([norm_vals], dtype=torch.float32).to(device)).tolist())
    else:
        x = np.array([0 for i in range(numFeatures)])
    data[k, 3*numFeatures:4*numFeatures] = x
    

print(len(TEST_FEATURES), data.shape)
test[TEST_FEATURES] = data

    
print()
print('New test shape:',test.shape)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
We are creating 800 features for 1 rows... 0 , 800 (1, 800)

New test shape: (1, 803)


In [22]:

# Columns to be excluded from scaling
excluded_columns = ['eeg_id', 'spectrogram_id', 'patient_id']

# Save the columns to be excluded
excluded_data = test[excluded_columns]

# DataFrame with only the columns to be scaled
features = test.drop(columns=excluded_columns)

# Initialize the StandardScaler
# scaler = StandardScaler()

# Fit the scaler to the features and transform them
features_scaled = scaler.transform(features)

# Create a DataFrame from the scaled features
features_scaled_df = pd.DataFrame(features_scaled, columns=features.columns)

# Concatenate the scaled features with the excluded columns
test_scaled_df = pd.concat([excluded_data.reset_index(drop=True),features_scaled_df,], axis=1)
test_scaled_df 


,eeg_id,spectrogram_id,patient_id,feature_0_alpha,feature_1_alpha,feature_2_alpha,feature_3_alpha,feature_4_alpha,feature_5_alpha,feature_6_alpha,...,feature_190_theta,feature_191_theta,feature_192_theta,feature_193_theta,feature_194_theta,feature_195_theta,feature_196_theta,feature_197_theta,feature_198_theta,feature_199_theta
0,3911565283,853520,6885,-0.086031,0.0,0.0,-0.08253,-0.090489,-0.080801,0.0,...,0.094635,0.0,0.0,0.0,0.0,0.590842,0.0,0.0,0.0,0.0


In [23]:
# INFER XGBOOST ON TEST
preds = []

for i in range(n_splits):
    print(i, ', ', end='')
    
    # Load the XGBoost model
    with open(f'XGBoost_f{i}.pkl', 'rb') as f:
        model = pickle.load(f)
    
    # Make predictions
    test_data_scaled = test_scaled_df[TEST_FEATURES]
    
    # data_imputed = imputer.fit_transform(test_data_scaled)
    
    pred = model.predict(test_data_scaled)
    pred[pred < 0] = 0
    pred = pred / np.sum(pred, axis=1).reshape(-1,1)
    preds.append(pred) 

# Average the predictions from each fold
pred = np.mean(preds, axis=0)
print('Test preds shape', pred.shape)

0 , 1 , 2 , 3 , 4 , Test preds shape (1, 6)


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFCE30"><b><span style='color:#FFFFFF'>12 |</span></b> <b>SUBMISSION</b></div>

In [24]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[LABEL_NAMES] = pred
sub.to_csv('submission.csv',index=False)
print('Submission shape',sub.shape)
sub.head()

Submission shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.105152,0.177444,0.0,0.110224,0.027732,0.579449


In [25]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float32